In [1]:
import jieba
import numpy as np

In [2]:
import json
from tqdm import tqdm
import os
from random import choice
from itertools import groupby
# from __future__ import print_function

In [3]:
def byteify(input):
    """把Unicode转换为encode"""
    if isinstance(input, dict):
        return {byteify(key):byteify(value) for key,value in input.items()}
        print("dic...dict")
    elif isinstance(input, list):
        return [byteify(element) for element in input]
        print("list...list...")
    elif isinstance(input, unicode):
        return input.encode('utf-8')
        print("unicode....unicode")
    else:
        return input
    

In [4]:
def load_data(data_directory):
    with open(data_directory) as f:
        train_data,dev_data,test_data = json.load(f)
#         train_data = byteify(train_data)
#         dev_data = byteify(dev_data)
#         test_data = byteify(test_data)
    print('traindata size:',len(train_data))
    print('devdata size:',len(dev_data))
    print('testdata size',len(test_data))
    return train_data,dev_data,test_data

In [5]:
train_data,dev_data,test_data = load_data('./data/all_data.json')
print(train_data[0])

traindata size: 84262
devdata size: 851
testdata size 852
{'text_id': '81967', 'text': '电影《淘金岁月》下载 急求', 'mention_data': [['电影', 0, '148097'], ['淘金岁月', 3, '50357']]}


In [6]:
print(train_data[0]['text'])

电影《淘金岁月》下载 急求


In [7]:
label_type = ['B','M','E','S']

In [8]:
import jieba

In [9]:
def generate_ner_data(data):
    char = []
    seg = []
    label = []
    ids = []
    kb_ids = []
    for i in tqdm(range(len(data))):
        tmp_text = list(data[i]['text'])
        tmp_label = ['O']*len(tmp_text)
        tmp_mention = data[i]['mention_data']
        kb_id = []
        for j in range(len(tmp_mention)):
            m,o,_id = tmp_mention[j]
            kb_id.append(_id)
            if len(m) > 1:
                tmp_label[o] = 'B'
                tmp_label[o+len(m)-1] = 'E'
                for j in range(o+1,o+len(m)-1,1):
                    tmp_label[j] = 'M'
            else:
                tmp_label[o] = 'S'
        seg.append(list(jieba.cut(data[i]['text'])))
        char.append(tmp_text)
        label.append(tmp_label)
        ids.append(data[i]['text_id'])
        kb_ids.append(kb_id)
    return char,seg,label,ids,kb_ids

In [10]:
print(dev_data[0])

{'text_id': '59668', 'text': '《铁梨花》,结局让人有着说不出的失落。 - 铁梨花', 'mention_data': [['铁梨花', 1, '143416'], ['结局', 6, '335878'], ['失落', 16, '286921'], ['铁梨花', 22, '143416']]}


In [11]:
dev_char,dev_seg,dev_label,dev_ids,dev_kb_ids  = generate_ner_data(dev_data)

  0%|          | 0/851 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.892 seconds.
Prefix dict has been built succesfully.
100%|██████████| 851/851 [00:01<00:00, 787.58it/s]


In [12]:
print(dev_data[0])
print(dev_char[0])
print(dev_seg[0])
print(dev_label[0])
print(dev_ids[0])
print(dev_kb_ids[0])

{'text_id': '59668', 'text': '《铁梨花》,结局让人有着说不出的失落。 - 铁梨花', 'mention_data': [['铁梨花', 1, '143416'], ['结局', 6, '335878'], ['失落', 16, '286921'], ['铁梨花', 22, '143416']]}
['《', '铁', '梨', '花', '》', ',', '结', '局', '让', '人', '有', '着', '说', '不', '出', '的', '失', '落', '。', ' ', '-', ' ', '铁', '梨', '花']
['《', '铁', '梨花', '》', ',', '结局', '让', '人', '有着', '说不出', '的', '失落', '。', ' ', '-', ' ', '铁', '梨花']
['O', 'B', 'M', 'E', 'O', 'O', 'B', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'E', 'O', 'O', 'O', 'O', 'B', 'M', 'E']
59668
['143416', '335878', '286921', '143416']


字典树 查找子单词

In [13]:
import collections
class TrieNode:
    # Initialize your data structure here.
    def __init__(self):
        self.children = collections.defaultdict(TrieNode)
        self.is_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        
        current = self.root
        for letter in word:
            current = current.children[letter]
        current.is_word = True

    def search(self, word):
        current = self.root
        for letter in word:
            current = current.children.get(letter)

            if current is None:
                return False
        return current.is_word

    def startsWith(self, prefix):
        current = self.root
        for letter in prefix:
            current = current.children.get(letter)
            if current is None:
                return False
        return True


    def enumerateMatch(self, word, space="_", backward=False):
        matched = []
        ## while len(word) > 1 does not keep character itself, while word keed character itself
        while len(word) > 1:
            if self.search(word):
                matched.append(space.join(word[:]))
            del word[-1]
        return matched

In [14]:
class Gazetteer:
    def __init__(self, lower):
        self.trie = Trie()
        self.ent2type = {} ## word list to type
        self.ent2id = {"<UNK>":0}   ## word list to id
        self.lower = lower
        self.space = ""

    def enumerateMatchList(self, word_list):
        if self.lower:
            word_list = [word.lower() for word in word_list]
        match_list = self.trie.enumerateMatch(word_list, self.space)
        return match_list

    def insert(self, word_list, source):
        if self.lower:
            word_list = [word.lower() for word in word_list]
        self.trie.insert(word_list)
        string = self.space.join(word_list)
        if string not in self.ent2type:
            self.ent2type[string] = source
        if string not in self.ent2id:
            self.ent2id[string] = len(self.ent2id)

    def searchId(self, word_list):
        if self.lower:
            word_list = [word.lower() for word in word_list]
        string = self.space.join(word_list)
        if string in self.ent2id:
            return self.ent2id[string]
        return self.ent2id["<UNK>"]

    def searchType(self, word_list):
        if self.lower:
            word_list = [word.lower() for word in word_list]
        string = self.space.join(word_list)
        if string in self.ent2type:
            return self.ent2type[string]
        print("Error in finding entity type at gazetteer.py, exit program! String:", string)
        exit(0)

    def size(self):
        return len(self.ent2type)

In [15]:
from KB import KB

In [16]:
kb_data = KB('./ccks2019_el/kb_data')

4389it [00:00, 20880.41it/s]

start loading kb_data...
construct id2kb dict...


399252it [00:18, 21367.61it/s]


construct kb2id dict...
KB DATA INFORMATION
TOKEN SIZE:303375
ID SIZE:399233
TYPE SIZE:51
PREDICATE SIZE:41841


In [17]:
gaz = Gazetteer(True)

In [18]:
for en in kb_data.kb:
    gaz.insert(en, "one_source")

In [19]:
print(gaz.ent2type[kb_data.kb[0]])

one_source


In [20]:
print(dev_data)

[{'text_id': '59668', 'text': '《铁梨花》,结局让人有着说不出的失落。 - 铁梨花', 'mention_data': [['铁梨花', 1, '143416'], ['结局', 6, '335878'], ['失落', 16, '286921'], ['铁梨花', 22, '143416']]}, {'text_id': '86064', 'text': '【《十一罗汉》】什么意思_英语《十一罗汉》的', 'mention_data': [['英语', 13, '274348']]}, {'text_id': '70831', 'text': '《冰火奇缘》开机 陈汉典笑称颜值堪比吴亦凡', 'mention_data': [['冰火奇缘', 1, '180506'], ['陈汉典', 9, '258143'], ['颜值', 14, '338974'], ['吴亦凡', 18, '385249']]}, {'text_id': '20876', 'text': '《我曾这样寂寞生活:辛波斯卡诗选2》的笔记-种种可能', 'mention_data': [['我曾这样寂寞生活:辛波斯卡诗选2', 1, '73452'], ['笔记', 19, '70791']]}, {'text_id': '1566', 'text': '一顾倾人城,再顾倾人国。 汉朝 李延年《佳人歌 ...', 'mention_data': [['城', 4, '52848'], ['国', 10, '210403'], ['汉朝', 13, '317957'], ['李延年', 16, '266721'], ['佳人歌', 20, '233904']]}, {'text_id': '54115', 'text': '求电视剧《外乡人》的电视剧音乐', 'mention_data': [['电视剧', 1, '202048'], ['外乡人', 5, '251457'], ['电视剧', 10, '202048'], ['音乐', 13, '130934']]}, {'text_id': '9107', 'text': '绿洲 影评╟ mtime时光网', 'mention_data': [['绿洲', 0, '159221'], ['影评', 3, '2226

In [21]:
kb_data.kb2id['中文']

['13267']

In [22]:
kb_data.kb2id['中']

['140079', '141476', '288316', '383623']

In [23]:
kb_data.kb2id['中文版']

['304975']

In [24]:
for i in range(len("铁梨花")):
    print(gaz.enumerateMatchList("铁梨花"[i:]))

['铁梨花']
['梨花']
[]


In [25]:
len_train = [len(x['text']) for x in train_data]
print(sum(len_train)/len(len_train))
print(max(len_train))

21.763416486672522
49


In [26]:
import matplotlib.pyplot as plt
# plt.plot(np.arange(len(len_train)),len_train)
plt.hist(len_train)
plt.show()

<Figure size 640x480 with 1 Axes>

In [27]:
import json
import os

class Alphabet:
    def __init__(self, name, label=False, keep_growing=True):
        self.__name = name
        self.UNKNOWN = "</unk>"
        self.label = label
        self.instance2index = {}
        self.instances = []
        self.keep_growing = keep_growing

        # Index 0 is occupied by default, all else following.
        self.default_index = 0
        self.next_index = 1
        if not self.label:
            self.add(self.UNKNOWN)

    def clear(self, keep_growing=True):
        self.instance2index = {}
        self.instances = []
        self.keep_growing = keep_growing

        # Index 0 is occupied by default, all else following.
        self.default_index = 0
        self.next_index = 1
        
    def add(self, instance):
        if instance not in self.instance2index:
            self.instances.append(instance)
            self.instance2index[instance] = self.next_index
            self.next_index += 1

    def get_index(self, instance):
        try:
            return self.instance2index[instance]
        except KeyError:
            if self.keep_growing:
                index = self.next_index
                self.add(instance)
                return index
            else:
                return self.instance2index[self.UNKNOWN]

    def get_instance(self, index):
        if index == 0:
            # First index is occupied by the wildcard element.
            return None
        try:
            return self.instances[index - 1]
        except IndexError:
            print('WARNING:Alphabet get_instance ,unknown instance, return the first label.')
            return self.instances[0]

    def size(self):
        # if self.label:
        #     return len(self.instances)
        # else:
        return len(self.instances) + 1

    def items(self):
        return self.instance2index.items()

    def enumerate_items(self, start=1):
        if start < 1 or start >= self.size():
            raise IndexError("Enumerate is allowed between [1 : size of the alphabet)")
        return zip(range(start, len(self.instances) + 1), self.instances[start - 1:])

    def close(self):
        self.keep_growing = False

    def open(self):
        self.keep_growing = True

    def get_content(self):
        return {'instance2index': self.instance2index, 'instances': self.instances}

    def from_json(self, data):
        self.instances = data["instances"]
        self.instance2index = data["instance2index"]

    def save(self, output_directory, name=None):
        """
        Save both alhpabet records to the given directory.
        :param output_directory: Directory to save model and weights.
        :param name: The alphabet saving name, optional.
        :return:
        """
        saving_name = name if name else self.__name
        try:
            json.dump(self.get_content(), open(os.path.join(output_directory, saving_name + ".json"), 'w'))
        except Exception as e:
            print("Exception: Alphabet is not saved: " % repr(e))

    def load(self, input_directory, name=None):
        """
        Load model architecture and weights from the give directory. This allow we use old models even the structure
        changes.
        :param input_directory: Directory to save model and weights
        :return:
        """
        loading_name = name if name else self.__name
        self.from_json(json.load(open(os.path.join(input_directory, loading_name + ".json"))))

In [28]:
import sys
import numpy as np
# from alphabet import Alphabet
NULLKEY = "-null-"

In [29]:
def normalize_word(word):
    # 数字归一化为0
    new_word = ""
    for char in word:
        if char.isdigit():
            new_word += '0'
        else:
            new_word += char
    return new_word

In [30]:
# def read_instance(input_file, word_alphabet, char_alphabet, label_alphabet, number_normalized,max_sent_length, char_padding_size=-1, char_padding_symbol = '</pad>'):
def read_instance(text,text_label,word_alphabet, char_alphabet, label_alphabet,\
                  number_normalized,max_sent_length, char_padding_size=-1, char_padding_symbol = '</pad>'):
#     in_lines = open(input_file,'r').readlines()
    assert(len(text)==len(text_label))
    instence_texts = []
    instence_Ids = []
    words = []
    chars = []
    labels = []
    word_Ids = []
    char_Ids = []
    label_Ids = []
#     for line in in_lines:
    for i in range(len(text)):
        print(i)
        for j in range(len(text[i])):
#         if len(line) > 2:
#             pairs = line.strip().split()
#             word = pairs[0].decode('utf-8')
            word = text[i][j]
            if number_normalized:
                word = normalize_word(word)
#             label = pairs[-1]
            label = text_label[i][j]
            words.append(word)
            labels.append(label)
            word_Ids.append(word_alphabet.get_index(word))
            label_Ids.append(label_alphabet.get_index(label))
            char_list = []
            char_Id = []
            for char in word:
                char_list.append(char)
            if char_padding_size > 0:
                char_number = len(char_list)
                if char_number < char_padding_size:
                    char_list = char_list + [char_padding_symbol]*(char_padding_size-char_number)
                assert(len(char_list) == char_padding_size)
            else:
                ### not padding
                pass
            for char in char_list:
                char_Id.append(char_alphabet.get_index(char))
            chars.append(char_list)
            char_Ids.append(char_Id)
        if (max_sent_length < 0) or (len(words) < max_sent_length):
            instence_texts.append([words, chars, labels])
            instence_Ids.append([word_Ids, char_Ids,label_Ids])
        words = []
        chars = []
        labels = []
        word_Ids = []
        char_Ids = []
        label_Ids = []
#         else:
#             if (max_sent_length < 0) or (len(words) < max_sent_length):
#                 instence_texts.append([words, chars, labels])
#                 instence_Ids.append([word_Ids, char_Ids,label_Ids])
#             words = []
#             chars = []
#             labels = []
#             word_Ids = []
#             char_Ids = []
#             label_Ids = []
    
    return instence_texts, instence_Ids

In [31]:
def read_seg_instance(text,text_label, word_alphabet, biword_alphabet, char_alphabet, label_alphabet, number_normalized, max_sent_length, char_padding_size=-1, char_padding_symbol = '</pad>'):
#     in_lines = open(input_file,'r').readlines()
    assert(len(text)==len(text_label))
    instence_texts = []
    instence_Ids = []
    words = []
    biwords = []
    chars = []
    labels = []
    word_Ids = []
    biword_Ids = []
    char_Ids = []
    label_Ids = []
    for i in range(len(text)):
        for j in range(len(text[i])):
#     for idx in xrange(len(in_lines)):
#         line = in_lines[idx]
#         if len(line) > 2:
#             pairs = line.strip().split()
#             word = pairs[0].decode('utf-8')
            word = text[i][j]
#             print(word)
            if number_normalized:
                word = normalize_word(word)
#             label = pairs[-1]
            label = text_label[i][j]
            words.append(word)
#             if idx < len(in_lines) -1 and len(in_lines[idx+1]) > 2:
#                 biword = word + in_lines[idx+1].strip().split()[0].decode('utf-8')
            if j < len(text[i]) - 1:
                biword = word + text[i][j+1]
            else:
                biword = word + NULLKEY
            biwords.append(biword)
            labels.append(label)
            word_Ids.append(word_alphabet.get_index(word))
            biword_Ids.append(biword_alphabet.get_index(biword))
            label_Ids.append(label_alphabet.get_index(label))
            char_list = []
            char_Id = []
            for char in word:
                char_list.append(char)
            if char_padding_size > 0:
                char_number = len(char_list)
                if char_number < char_padding_size:
                    char_list = char_list + [char_padding_symbol]*(char_padding_size-char_number)
                assert(len(char_list) == char_padding_size)
            else:
                ### not padding
                pass
            for char in char_list:
                char_Id.append(char_alphabet.get_index(char))
            chars.append(char_list)
            char_Ids.append(char_Id)
        if (max_sent_length < 0) or (len(words) < max_sent_length):
            instence_texts.append([words, biwords, chars, labels])
            instence_Ids.append([word_Ids, biword_Ids, char_Ids,label_Ids])
        words = []
        biwords = []
        chars = []
        labels = []
        word_Ids = []
        biword_Ids = []
        char_Ids = []
        label_Ids = []
#         else:
#             if (max_sent_length < 0) or (len(words) < max_sent_length):
#                 instence_texts.append([words, biwords, chars, labels])
#                 instence_Ids.append([word_Ids, biword_Ids, char_Ids,label_Ids])
#             words = []
#             biwords = []
#             chars = []
#             labels = []
#             word_Ids = []
#             biword_Ids = []
#             char_Ids = []
#             label_Ids = []

    return instence_texts, instence_Ids

In [32]:
# a,b = read_seg_instance(dev_x,dev_y, data.word_alphabet, data.biword_alphabet, data.char_alphabet, data.label_alphabet, data.number_normalized, data.MAX_SENTENCE_LENGTH)

In [33]:
# print(a[0])
# print(b[0])

In [34]:
def read_instance_with_gaz(text,text_label, gaz, word_alphabet, biword_alphabet, char_alphabet, gaz_alphabet, label_alphabet, number_normalized, max_sent_length, char_padding_size=-1, char_padding_symbol = '</pad>'):
#     in_lines = open(input_file,'r').readlines()
    assert(len(text)==len(text_label))
    instence_texts = []
    instence_Ids = []
    words = []
    biwords = []
    chars = []
    labels = []
    word_Ids = []
    biword_Ids = []
    char_Ids = []
    label_Ids = []
    for i in range(len(text)):
        for j in range(len(text[i])):
#     for idx in xrange(len(in_lines)):
#         line = in_lines[idx]
#         if len(line) > 2:
#             pairs = line.strip().split()
#             word = pairs[0].decode('utf-8')
            word = text[i][j]
            if number_normalized:
                word = normalize_word(word)
#             label = pairs[-1]
            label = text_label[i][j]
#             if idx < len(in_lines) -1 and len(in_lines[idx+1]) > 2:
#                 biword = word + in_lines[idx+1].strip().split()[0].decode('utf-8')
            if j < len(text[i]) - 1:
                biword = word + text[i][j+1]
            else:
                biword = word + NULLKEY
            biwords.append(biword)
            words.append(word)
            labels.append(label)
            word_Ids.append(word_alphabet.get_index(word))
            biword_Ids.append(biword_alphabet.get_index(biword))
            label_Ids.append(label_alphabet.get_index(label))
            char_list = []
            char_Id = []
            for char in word:
                char_list.append(char)
            if char_padding_size > 0:
                char_number = len(char_list)
                if char_number < char_padding_size:
                    char_list = char_list + [char_padding_symbol]*(char_padding_size-char_number)
                assert(len(char_list) == char_padding_size)
            else:
                ### not padding
                pass
            for char in char_list:
                char_Id.append(char_alphabet.get_index(char))
            chars.append(char_list)
            char_Ids.append(char_Id)

#         else:
#         print(len(words))
        if ((max_sent_length < 0) or (len(words) < max_sent_length)) and (len(words)>0):
#             print(2)
            gazs = []
            gaz_Ids = []
            w_length = len(words)
            # print sentence 
            # for w in words:
            #     print w," ",
            # print
            for idx in range(w_length):
                matched_list = gaz.enumerateMatchList(words[idx:])
                matched_length = [len(a) for a in matched_list]
                # print idx,"----------"
                # print "forward...feed:","".join(words[idx:])
                # for a in matched_list:
                #     print a,len(a)," ",
                # print

                # print matched_length

                gazs.append(matched_list)
                matched_Id  = [gaz_alphabet.get_index(entity) for entity in matched_list]
                if matched_Id:
                    gaz_Ids.append([matched_Id, matched_length])
                else:
                    gaz_Ids.append([])

            instence_texts.append([words, biwords, chars, gazs, labels])
            instence_Ids.append([word_Ids, biword_Ids, char_Ids, gaz_Ids, label_Ids])
        words = []
        biwords = []
        chars = []
        labels = []
        word_Ids = []
        biword_Ids = []
        char_Ids = []
        label_Ids = []
        gazs = []
        gaz_Ids = []
    return instence_texts, instence_Ids

In [35]:
# a,b = read_instance_with_gaz(dev_char,dev_label, data.kb, data.word_alphabet, data.biword_alphabet, data.char_alphabet, data.gaz_alphabet,  data.label_alphabet, data.number_normalized, data.MAX_SENTENCE_LENGTH)

In [36]:
# print(a[0])

In [37]:
# print(b[0])

In [54]:
def build_pretrain_embedding(embedding_path, word_alphabet, embedd_dim=100, norm=True):    
    embedd_dict = dict()
    if embedding_path != None:
        embedd_dict, embedd_dim = load_pretrain_emb(embedding_path)
    scale = np.sqrt(3.0 / embedd_dim)
    pretrain_emb = np.empty([word_alphabet.size(), embedd_dim])
    perfect_match = 0
    case_match = 0
    not_match = 0
    for word, index in word_alphabet.items():
        if word in embedd_dict:
            if norm:
                pretrain_emb[index,:] = norm2one(embedd_dict[word])
            else:
                pretrain_emb[index,:] = embedd_dict[word]
            perfect_match += 1
        elif word.lower() in embedd_dict:
            if norm:
                pretrain_emb[index,:] = norm2one(embedd_dict[word.lower()])
            else:
                pretrain_emb[index,:] = embedd_dict[word.lower()]
            case_match += 1
        else:
            pretrain_emb[index,:] = np.random.uniform(-scale, scale, [1, embedd_dim])
            not_match += 1
    pretrained_size = len(embedd_dict)
    print("Embedding:\n     pretrain word:%s, prefect match:%s, case_match:%s, oov:%s, oov%%:%s"%(pretrained_size, perfect_match, case_match, not_match, (not_match+0.)/word_alphabet.size()))
    return pretrain_emb, embedd_dim

In [39]:
def norm2one(vec):
    root_sum_square = np.sqrt(np.sum(np.square(vec)))
    return vec/root_sum_square

def load_pretrain_emb(embedding_path):
    embedd_dim = -1
    embedd_dict = dict()
    with open(embedding_path, 'r') as file:
        for line in file:
            line = line.strip()
            if len(line) == 0:
                continue
            tokens = line.split()
            if embedd_dim < 0:
                embedd_dim = len(tokens) - 1
            else:
                assert (embedd_dim + 1 == len(tokens))
            embedd = np.empty([1, embedd_dim])
            embedd[:] = tokens[1:]
            embedd_dict[tokens[0].decode('utf-8')] = embedd
    return embedd_dict, embedd_dim

In [40]:
# data.kb

In [41]:
START = "</s>"
UNKNOWN = "</unk>"
PADDING = "</pad>"
NULLKEY = "-null-"

In [42]:
class Data(object):
    def __init__(self): 
        self.MAX_SENTENCE_LENGTH = 50
        self.MAX_WORD_LENGTH = -1
        self.number_normalized = True
        self.norm_word_emb = True
        self.norm_biword_emb = True
        self.norm_gaz_emb = False
        self.word_alphabet = Alphabet('word')
        self.biword_alphabet = Alphabet('biword')
        self.char_alphabet = Alphabet('character')
        # self.word_alphabet.add(START)
        # self.word_alphabet.add(UNKNOWN)
        # self.char_alphabet.add(START)
        # self.char_alphabet.add(UNKNOWN)
        # self.char_alphabet.add(PADDING)
        self.label_alphabet = Alphabet('label', True)
        
        self.gaz_lower = False
        self.gaz = Gazetteer(self.gaz_lower)
        self.gaz_alphabet = Alphabet('gaz')
        self.HP_fix_gaz_emb = False
        self.HP_use_gaz = True
        
        self.kb_lower = False
        self.kb = Gazetteer(self.kb_lower)
        self.kb_alphabet = Alphabet('kb')
        self.norm_kb_emb = False
        self.HP_fix_kb_emb = False
        self.HP_use_kb = True        

        self.tagScheme = "NoSeg"
        self.char_features = "LSTM" 

        self.train_texts = []
        self.dev_texts = []
        self.test_texts = []
        self.raw_texts = []

        self.train_Ids = []
        self.dev_Ids = []
        self.test_Ids = []
        self.raw_Ids = []
        self.use_bigram = True
        self.word_emb_dim = 50
        self.biword_emb_dim = 50
        self.char_emb_dim = 30
        self.gaz_emb_dim = 50
        self.kb_emb_dim = 50
        self.gaz_dropout = 0.5
        self.pretrain_word_embedding = None
        self.pretrain_biword_embedding = None
        self.pretrain_gaz_embedding = None
        self.label_size = 0
        self.word_alphabet_size = 0
        self.biword_alphabet_size = 0
        self.char_alphabet_size = 0
        self.label_alphabet_size = 0
        ### hyperparameters
        self.HP_iteration = 100
        self.HP_batch_size = 10
        self.HP_char_hidden_dim = 50
        self.HP_hidden_dim = 200
        self.HP_dropout = 0.5
        self.HP_lstm_layer = 1
        self.HP_bilstm = True
        self.HP_use_char = False
        self.HP_gpu = False
        self.HP_lr = 0.015
        self.HP_lr_decay = 0.05
        self.HP_clip = 5.0
        self.HP_momentum = 0
    def show_data_summary(self):
        print("DATA SUMMARY START:")
        print("     Tag          scheme: %s"%(self.tagScheme))
        print("     MAX SENTENCE LENGTH: %s"%(self.MAX_SENTENCE_LENGTH))
        print("     MAX   WORD   LENGTH: %s"%(self.MAX_WORD_LENGTH))
        print("     Number   normalized: %s"%(self.number_normalized))
        print("     Use          bigram: %s"%(self.use_bigram))
        print("     Word  alphabet size: %s"%(self.word_alphabet_size))
        print("     Biword alphabet size: %s"%(self.biword_alphabet_size))
        print("     Char  alphabet size: %s"%(self.char_alphabet_size))
        print("     Gaz   alphabet size: %s"%(self.gaz_alphabet.size()))
        print("     KB   alphabet size: %s"%(self.kb_alphabet.size()))
        print("     Label alphabet size: %s"%(self.label_alphabet_size))
        print("     Word embedding size: %s"%(self.word_emb_dim))
        print("     Biword embedding size: %s"%(self.biword_emb_dim))
        print("     Char embedding size: %s"%(self.char_emb_dim))
        print("     Gaz embedding size: %s"%(self.gaz_emb_dim))
        print("     KB embedding size: %s"%(self.kb_emb_dim))
        print("     Norm     word   emb: %s"%(self.norm_word_emb))
        print("     Norm     biword emb: %s"%(self.norm_biword_emb))
        print("     Norm     gaz    emb: %s"%(self.norm_gaz_emb))
        print("     Norm   gaz  dropout: %s"%(self.gaz_dropout))
        print("     Train instance number: %s"%(len(self.train_texts)))
        print("     Dev   instance number: %s"%(len(self.dev_texts)))
        print("     Test  instance number: %s"%(len(self.test_texts)))
        print("     Raw   instance number: %s"%(len(self.raw_texts)))
        print("     Hyperpara  iteration: %s"%(self.HP_iteration))
        print("     Hyperpara  batch size: %s"%(self.HP_batch_size))
        print("     Hyperpara          lr: %s"%(self.HP_lr))
        print("     Hyperpara    lr_decay: %s"%(self.HP_lr_decay))
        print("     Hyperpara     HP_clip: %s"%(self.HP_clip))
        print("     Hyperpara    momentum: %s"%(self.HP_momentum))
        print("     Hyperpara  hidden_dim: %s"%(self.HP_hidden_dim))
        print("     Hyperpara     dropout: %s"%(self.HP_dropout))
        print("     Hyperpara  lstm_layer: %s"%(self.HP_lstm_layer))
        print("     Hyperpara      bilstm: %s"%(self.HP_bilstm))
        print("     Hyperpara         GPU: %s"%(self.HP_gpu))
        print("     Hyperpara     use_gaz: %s"%(self.HP_use_gaz))
        print("     Hyperpara fix gaz emb: %s"%(self.HP_fix_gaz_emb))
        print("     Hyperpara    use_char: %s"%(self.HP_use_char))
        if self.HP_use_char:
            print("             Char_features: %s"%(self.char_features))
        print("DATA SUMMARY END.")
        sys.stdout.flush()
        
    def refresh_label_alphabet(self, labels):
        old_size = self.label_alphabet_size
        self.label_alphabet.clear(True)
#         in_lines = open(input_file,'r').readlines()
#         for line in in_lines:
#             if len(line) > 2:
        for i in range(len(labels)):
            for j in range(len(labels[i])):
#                 pairs = line.strip().split()
#                 label = pairs[-1]
                label = labels[i][j]
                self.label_alphabet.add(label)
        self.label_alphabet_size = self.label_alphabet.size()
        startS = False
        startB = False
        for label,_ in self.label_alphabet.items():
            if "S" in label.upper():
                startS = True
            elif "B" in label.upper():
                startB = True
        if startB:
            if startS:
                self.tagScheme = "BMES"
            else:
                self.tagScheme = "BIO"
        self.fix_alphabet()
        print("Refresh label alphabet finished: old:%s -> new:%s"%(old_size, self.label_alphabet_size))
    
    def build_alphabet(self,texts,labels):
#         in_lines = open(input_file,'r').readlines()
        assert(len(texts)==len(labels))
        idx = 0
        for i in range(len(texts)):
#             if len(text)
#             for k in range(len(labels[i])):
#                 label = labels[i][k]
#                 self.label_alphabet.add(label)
            for j in range(len(texts[i])):
                word = texts[i][j]
                if self.number_normalized:
                    word = normalize_word(word)
                label = labels[i][j]
                self.label_alphabet.add(label)                
                
                self.word_alphabet.add(word)
#                 if j < len(text[i]) - 1 and len(text[i]) > 2:
                if j < len(texts[i]) - 1:
                    biword = word + texts[i][j+1]
                else:
                    biword = word + NULLKEY
                self.biword_alphabet.add(biword)
                for char in word:
                    self.char_alphabet.add(char)
        self.word_alphabet_size = self.word_alphabet.size()
        self.biword_alphabet_size = self.biword_alphabet.size()
        self.char_alphabet_size = self.char_alphabet.size()
        self.label_alphabet_size = self.label_alphabet.size()
        startS = False
        startB = False
        for label,_ in self.label_alphabet.items():
            if "S" in label.upper():
                startS = True
            elif "B" in label.upper():
                startB = True
        if startB:
            if startS:
                self.tagScheme = "BMES"
            else:
                self.tagScheme = "BIO"
                
    def build_gaz_file(self, gaz_file):
        ## build gaz file,initial read gaz embedding file
        if gaz_file:
            fins = open(gaz_file, 'r').readlines()
            for fin in fins:
                fin = fin.strip().split()[0].decode('utf-8')
                if fin:
                    self.gaz.insert(fin, "one_source")
            print ("Load gaz file: ", gaz_file, " total size:", self.gaz.size())
        else:
            print ("Gaz file is None, load nothing")
    
    def build_kb(self,kb):
        if kb:
            for en in kb:
                self.kb.insert(en, "one_source")
            print("KB total size:",self.kb.size())
        else:
            print("KB file is None, load nothing")
    
    def build_all_alphabet(self,text,label):
        word_list = []
        for i in range(len(text)):
            for j in range(len(text[i])):
                word = text[i][j]
                if self.number_normalized:
                    word = normalize_word(word)
                word_list.append(word)
            else:
                w_length = len(word_list)
                for idx in range(w_length):
                    if self.gaz.size():
                        matched_gaz = self.gaz.enumerateMatchList(word_list[idx:])
                        for entity in matched_gaz:
                            # print entity, self.gaz.searchId(entity),self.gaz.searchType(entity)
                            self.gaz_alphabet.add(entity)
                    if self.kb.size():
                        matched_kb = self.kb.enumerateMatchList(word_list[idx:])
                        for entity in matched_kb:
                            # print entity, self.gaz.searchId(entity),self.gaz.searchType(entity)
                            self.kb_alphabet.add(entity)
                word_list = []
        print("gaz alphabet size:", self.gaz_alphabet.size()) 
        print("kb alphabet size:", self.kb_alphabet.size()) 


    def fix_alphabet(self):
        self.word_alphabet.close()
        self.biword_alphabet.close()
        self.char_alphabet.close()
        self.label_alphabet.close() 
        self.gaz_alphabet.close()  
        self.kb_alphabet.close()
    
    
    
    def build_word_pretrain_emb(self, emb_path):
        print ("build word pretrain emb...")
        self.pretrain_word_embedding, self.word_emb_dim = build_pretrain_embedding(emb_path, self.word_alphabet, self.word_emb_dim, self.norm_word_emb)

    def build_biword_pretrain_emb(self, emb_path):
        print ("build biword pretrain emb...")
        self.pretrain_biword_embedding, self.biword_emb_dim = build_pretrain_embedding(emb_path, self.biword_alphabet, self.biword_emb_dim, self.norm_biword_emb)

    def build_gaz_pretrain_emb(self, emb_path):
        print ("build gaz pretrain emb...")
        self.pretrain_gaz_embedding, self.gaz_emb_dim = build_pretrain_embedding(emb_path, self.gaz_alphabet,  self.gaz_emb_dim, self.norm_gaz_emb)
        
    def build_kb_pretrain_emb(self, emb_path):
        print ("build gaz pretrain emb...")
        self.pretrain_kb_embedding, self.kb_emb_dim = build_pretrain_embedding(emb_path, self.kb_alphabet,  self.kb_emb_dim, self.norm_kb_emb)

    def generate_instance(self, texts,labels, name):
        self.fix_alphabet()
        if name == "train":
            self.train_texts, self.train_Ids = read_seg_instance(texts,labels, self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "dev":
            self.dev_texts, self.dev_Ids = read_seg_instance(texts,labels, self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "test":
            self.test_texts, self.test_Ids = read_seg_instance(texts,labels, self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "raw":
            self.raw_texts, self.raw_Ids = read_seg_instance(texts,labels, self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        else:
            print("Error: you can only generate train/dev/test instance! Illegal input:%s"%(name))


    def generate_instance_with_gaz(self, texts,labels, name):
        self.fix_alphabet()
        if name == "train":
            self.train_texts, self.train_Ids = read_instance_with_gaz(texts,labels, self.gaz, self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.gaz_alphabet,  self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "dev":
            self.dev_texts, self.dev_Ids = read_instance_with_gaz(texts,labels, self.gaz,self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.gaz_alphabet,  self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "test":
            self.test_texts, self.test_Ids = read_instance_with_gaz(texts,labels, self.gaz, self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.gaz_alphabet,  self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "raw":
            self.raw_texts, self.raw_Ids = read_instance_with_gaz(texts,labels, self.gaz, self.word_alphabet,self.biword_alphabet, self.char_alphabet, self.gaz_alphabet,  self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        else:
            print("Error: you can only generate train/dev/test instance! Illegal input:%s"%(name))
    
    def generate_instance_with_kb(self, texts,labels, name):
        self.fix_alphabet()
        if name == "train":
            self.train_texts, self.train_Ids = read_instance_with_gaz(texts,labels, self.kb, self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.kb_alphabet,  self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "dev":
            self.dev_texts, self.dev_Ids = read_instance_with_gaz(texts,labels, self.kb,self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.kb_alphabet,  self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "test":
            self.test_texts, self.test_Ids = read_instance_with_gaz(texts,labels, self.kb, self.word_alphabet, self.biword_alphabet, self.char_alphabet, self.kb_alphabet,  self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        elif name == "raw":
            self.raw_texts, self.raw_Ids = read_instance_with_gaz(texts,labels, self.kb, self.word_alphabet,self.biword_alphabet, self.char_alphabet, self.kb_alphabet,  self.label_alphabet, self.number_normalized, self.MAX_SENTENCE_LENGTH)
        else:
            print("Error: you can only generate train/dev/test instance! Illegal input:%s"%(name))


    def write_decoded_results(self, output_file, predict_results, name):
        fout = open(output_file,'w')
        sent_num = len(predict_results)
        content_list = []
        if name == 'raw':
            content_list = self.raw_texts
        elif name == 'test':
            content_list = self.test_texts
        elif name == 'dev':
            content_list = self.dev_texts
        elif name == 'train':
            content_list = self.train_texts
        else:
            print("Error: illegal name during writing predict result, name should be within train/dev/test/raw !")
        assert(sent_num == len(content_list))
        for idx in range(sent_num):
            sent_length = len(predict_results[idx])
            for idy in range(sent_length):
                ## content_list[idx] is a list with [word, char, label]
                fout.write(content_list[idx][0][idy].encode('utf-8') + " " + predict_results[idx][idy] + '\n')

            fout.write('\n')
        fout.close()
        print("Predict %s result has been written into file. %s"%(name, output_file))

!!!!!词典的内容应为kb+训练集分词的结果

In [43]:
data = Data()
# data.show_data_summary()

In [44]:
print(data.kb_alphabet.instances)

['</unk>']


In [46]:
data.build_alphabet(dev_char,dev_label)
data.show_data_summary()

DATA SUMMARY START:
     Tag          scheme: BMES
     MAX SENTENCE LENGTH: 50
     MAX   WORD   LENGTH: -1
     Number   normalized: True
     Use          bigram: True
     Word  alphabet size: 2024
     Biword alphabet size: 11161
     Char  alphabet size: 2024
     Gaz   alphabet size: 2
     KB   alphabet size: 2
     Label alphabet size: 6
     Word embedding size: 50
     Biword embedding size: 50
     Char embedding size: 30
     Gaz embedding size: 50
     KB embedding size: 50
     Norm     word   emb: True
     Norm     biword emb: True
     Norm     gaz    emb: False
     Norm   gaz  dropout: 0.5
     Train instance number: 0
     Dev   instance number: 0
     Test  instance number: 0
     Raw   instance number: 0
     Hyperpara  iteration: 100
     Hyperpara  batch size: 10
     Hyperpara          lr: 0.015
     Hyperpara    lr_decay: 0.05
     Hyperpara     HP_clip: 5.0
     Hyperpara    momentum: 0
     Hyperpara  hidden_dim: 200
     Hyperpara     dropout: 0.5
     Hyp

In [47]:
data.refresh_label_alphabet(dev_label)

Refresh label alphabet finished: old:6 -> new:6


In [48]:
data.build_kb(kb_data.kb)
data.build_all_alphabet(dev_char,dev_label)

KB total size: 303375
gaz alphabet size: 2
kb alphabet size: 3761


In [49]:
data.generate_instance_with_kb(dev_char,dev_label,'train')

In [50]:
print(data.train_Ids[1][-2])

[[], [], [[6, 7], [4, 2]], [], [[8], [2]], [], [], [], [[9], [2]], [], [[10], [2]], [], [], [[11], [2]], [], [], [[6, 7], [4, 2]], [], [[8], [2]], [], [], []]


In [51]:
print(data.train_texts[1][-2])

[[], [], ['十一罗汉', '十一'], [], ['罗汉'], [], [], [], ['什么'], [], ['意思'], [], [], ['英语'], [], [], ['十一罗汉', '十一'], [], ['罗汉'], [], [], []]


In [55]:
data.build_kb_pretrain_emb(None)

build gaz pretrain emb...
Embedding:
     pretrain word:0, prefect match:0, case_match:0, oov:3760, oov%:0.999734113267748


In [ ]:
print(data.label_alphabet.instances)

In [ ]:
data.build_alphabet(train_file)
data.build_alphabet(dev_file)
data.build_alphabet(test_file)
data.build_gaz_file(gaz_file)
data.build_gaz_alphabet(train_file)
data.build_gaz_alphabet(dev_file)
data.build_gaz_alphabet(test_file)
data.fix_alphabet()
return data